In [43]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [44]:
import os

DIR = os.path.abspath("..\\..\\")
DATA_DIR = DIR + "\\input\\qmdata0819"

DATA_DIR

'd:\\OneDrive\\0_caijia\\1_code\\python\\yammi_utils\\yammi_utils\\input\\qmdata0819'

## 读取数据集

In [45]:
read_all_csv = open(DATA_DIR + "\\all.csv", "r" , encoding="utf8")

data_ = []
for line in read_all_csv:
    line = line.strip()
    time_, item_ = line.split(',')
    data_.append([time_, item_])

# 第一行是表格的标题栏，pop掉
data_.pop(0)

['时间', '记录']

为 data_ 增加第三维内容。
第三维内容是时间戳，来自每条记录的时间文本

In [46]:
import time

for each in data_:
    each.append(int(time.mktime(time.strptime(each[0], "%Y-%m-%d %H:%M:%S"))))

len(data_)

82092

根据时间戳，分析数据的段落。前后2条数据的时间戳间隔不为58的，都视为段落分割点。

In [47]:
segment = []
all_segs = []
for i in range(len(data_)):
    segment.append(data_[i])
    if i<len(data_)-1 and i > 0 and data_[i+1][2] - data_[i][2] != 58:
        all_segs.append(segment)
        segment = []

all_segs.append(segment)

对于每一个段落，读取它的第0 - 20条数据，作为新的数据段落的起点。把它对应的第21条数据作为相应的结果。依次逐条推进，形成新的数据集段落列表 all_rows。

In [48]:
row_ = []
all_rows = []
for each in all_segs:
    for i in range(20,len(each)-1):
        list_ = each[i-20:i]
        row_ = [list_, each[i]]
        all_rows.append(row_)

len(all_rows)

79782

为 all_rows 里的每一条数据，增加一项统计数据，计算这一条数据里的20条历史记录，各物品对应的统计量。

In [49]:
item_names = ["钢琴", "小提琴", "吉他", "贝斯", "架子鼓", "竖琴", "萨克斯风", "圆号"]
def count_items(a_list:list):
    items = []
    # each 是每个回合的数据
    # each[1]是名字
    for each in a_list:
        items.append(each[1])
    
    count_result = []
    for n in item_names:
        count_result.append(items.count(n))
    
    # count_result是每一行数据的20个回合里，每个物品数量的统计
    return count_result

# each 是20个回合的出货数据 + 第21回合的出货数据
# each[0] 是20回合的结果
history = []
for each in all_rows:
    history.append(count_items(each[0]))

len(history)
history[-1]

[6, 3, 6, 2, 3, 0, 0, 0]

In [50]:
merge_data = []
for i in range(len(history)):
    merge_data.append([all_rows[i][0], all_rows[i][1], history[i]])

merge_data[-1]

[[['2022-11-03 10:49:04', '钢琴', 1667443744],
  ['2022-11-03 10:50:02', '小提琴', 1667443802],
  ['2022-11-03 10:51:00', '吉他', 1667443860],
  ['2022-11-03 10:51:58', '架子鼓', 1667443918],
  ['2022-11-03 10:52:56', '吉他', 1667443976],
  ['2022-11-03 10:53:54', '钢琴', 1667444034],
  ['2022-11-03 10:54:52', '钢琴', 1667444092],
  ['2022-11-03 10:55:50', '钢琴', 1667444150],
  ['2022-11-03 10:56:48', '吉他', 1667444208],
  ['2022-11-03 10:57:46', '吉他', 1667444266],
  ['2022-11-03 10:58:44', '小提琴', 1667444324],
  ['2022-11-03 10:59:42', '架子鼓', 1667444382],
  ['2022-11-03 11:00:40', '小提琴', 1667444440],
  ['2022-11-03 11:01:38', '钢琴', 1667444498],
  ['2022-11-03 11:02:36', '吉他', 1667444556],
  ['2022-11-03 11:03:34', '贝斯', 1667444614],
  ['2022-11-03 11:04:32', '吉他', 1667444672],
  ['2022-11-03 11:05:30', '钢琴', 1667444730],
  ['2022-11-03 11:06:28', '贝斯', 1667444788],
  ['2022-11-03 11:07:26', '架子鼓', 1667444846]],
 ['2022-11-03 11:08:24', '小提琴', 1667444904],
 [6, 3, 6, 2, 3, 0, 0, 0]]

## 构建数据集
merge_data 里的 each ，each[0]里的每个 item，抽取 item[1]，即物品的名字

In [51]:
import numpy as np
items_name_20 = []
item_result = ''
count = []
data_for_pd = []
for each in merge_data:
    item_result = each[1][1]
    count = each[2]
    if item_result in item_names[:4]:
        if count[item_names.index(item_result)] > np.mean(count[:4]):
            result = '高'
        elif count[item_names.index(item_result)] < np.mean(count[:4]):
            result = '低'
        else:
            result = '平'
    elif item_result in item_names[-4:]:
        if count[item_names.index(item_result)] > np.mean(count[-4:]):
            result = '高'
        elif count[item_names.index(item_result)] < np.mean(count[-4:]):
            result = '低'
        else:
            result = '平'
    for item in each[0]:
        items_name_20.append(item[1])
    high_low = []
    for item in items_name_20:
        if item in item_names[:4]:
            if count[item_names.index(item)] > np.mean(count[:4]):
                high_low.append('高')
            elif count[item_names.index(item)] < np.mean(count[:4]):
                high_low.append('低')
            else:
                high_low.append('平')
        elif item in item_names[-4:]:
            if count[item_names.index(item)] > np.mean(count[-4:]):
                high_low.append('高')
            elif count[item_names.index(item)] < np.mean(count[-4:]):
                high_low.append('低')
            else:
                high_low.append('平')
    data_for_pd.append([items_name_20,high_low, count, item_result, result])
    items_name_20 = []


data_for_pd[-1]

[['钢琴',
  '小提琴',
  '吉他',
  '架子鼓',
  '吉他',
  '钢琴',
  '钢琴',
  '钢琴',
  '吉他',
  '吉他',
  '小提琴',
  '架子鼓',
  '小提琴',
  '钢琴',
  '吉他',
  '贝斯',
  '吉他',
  '钢琴',
  '贝斯',
  '架子鼓'],
 ['高',
  '低',
  '高',
  '高',
  '高',
  '高',
  '高',
  '高',
  '高',
  '高',
  '低',
  '高',
  '低',
  '高',
  '高',
  '低',
  '高',
  '高',
  '低',
  '高'],
 [6, 3, 6, 2, 3, 0, 0, 0],
 '小提琴',
 '低']

In [55]:
a_1_4 = []
for each in data_for_pd:
    a_1_4.append(each[1]+[each[4]])

a_1_4[-2:]

[['低',
  '高',
  '低',
  '高',
  '高',
  '高',
  '高',
  '高',
  '高',
  '高',
  '高',
  '低',
  '高',
  '低',
  '高',
  '高',
  '低',
  '高',
  '高',
  '低',
  '高'],
 ['高',
  '低',
  '高',
  '高',
  '高',
  '高',
  '高',
  '高',
  '高',
  '高',
  '低',
  '高',
  '低',
  '高',
  '高',
  '低',
  '高',
  '高',
  '低',
  '高',
  '低']]

In [54]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [56]:
pd_1_4 = pd.DataFrame(a_1_4,columns=['col_1','col_2','col_3','col_4','col_5','col_6','col_7','col_8','col_9','col_10','col_11','col_12','col_13','col_14','col_15','col_16','col_17','col_18','col_19','col_20','result',])

pd_1_4.head()